In [1]:
import re
import os
import subprocess
import tempfile
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

/home/ykwang/.conda/envs/ykwang_llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# -------------------------------
# Load the APPS dataset and filter for introductory problems.
apps_dataset = load_dataset("codeparrot/apps")
train_dataset = apps_dataset["train"]
intro_train_dataset = train_dataset.filter(lambda example: example["difficulty"] == "introductory")

# -------------------------------
# Load the code generation model and tokenizer.
code_model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
code_tokenizer = AutoTokenizer.from_pretrained(code_model_name)
code_model = AutoModelForCausalLM.from_pretrained(code_model_name, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.15s/it]


In [3]:
def substitute_numbers(question):
    return re.sub(r'\d+', lambda x: str(int(x.group()) + 1), question)

# -------------------------------
# Build a prompt for code generation using a given question.
def build_code_prompt(question):
    return f"""
Please generate a complete Python script that solves the following problem. Enclose your final code within ```python and ``` markers. Please ensure that your final code includes a main block to read input and print the result. Also, ensure that your entire output does not exceed 1024 tokens.

Problem Description:
{question}

Generated Code:
"""

In [4]:
# -------------------------------
# Extract code from generated text (expects code to be between ```python and ```).
def extract_code(generated_text: str) -> str:
    pattern = r"```python\s*(.*?)\s*```"
    matches = re.findall(pattern, generated_text, re.DOTALL)
    if matches:
        return matches[-1].strip()
    else:
        return None

# -------------------------------
# Generate code using the code generation model.
def generate_code(prompt):
    inputs = code_tokenizer(prompt, return_tensors="pt").to(code_model.device)
    outputs = code_model.generate(
        inputs.input_ids,
        max_length=1024,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    generated_text = code_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return extract_code(generated_text)


In [5]:
# -------------------------------
# Run the generated code and record output.
def run_generated_code(code, io_pair):
    inputs_list = io_pair.get("inputs", [])
    recorded_outputs = []
    
    for inp in inputs_list:
        with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as tmp:
            tmp.write(code)
            tmp_filename = tmp.name
        try:
            result = subprocess.run(
                ["python", tmp_filename],
                input=inp,
                text=True,
                capture_output=True,
                timeout=10
            )
            recorded_outputs.append({"input": inp, "output": result.stdout.strip()})
        except Exception as e:
            recorded_outputs.append({"input": inp, "error": str(e)})
        finally:
            os.remove(tmp_filename)
    
    return recorded_outputs

In [ ]:
# -------------------------------
# Main procedure:
NUM_SAMPLES_ORIGINAL = 10
NUM_REPHRASES = 3
NUM_SAMPLES_PER_REPHRASE = 10

problem = intro_train_dataset[17]
io_pair = eval(problem["input_output"])
original_question = problem['question']

final_results = []

# ---- Test on the original question ----
print("\n=== Testing on Original Question ===")
original_prompt = build_code_prompt(original_question)
print(original_prompt)
os._exit()
original_results = []
for s in range(NUM_SAMPLES_ORIGINAL):
    print(f"\n--- Original Sample {s+1} ---")
    code = generate_code(original_prompt)
    print("Generated Code:\n", code, "\n")
    if code:
        recorded_outputs = run_generated_code(code, io_pair)
        original_results.append({"code": code, "recorded_outputs": recorded_outputs})
    else:
        original_results.append({"code": None, "recorded_outputs": []})
final_results.append(original_results)

# ---- Test on rephrased questions ----
for r in range(NUM_REPHRASES):
    print(f"\n=== Rephrase {r+1} ===")
    rephrased_question = substitute_numbers(original_question)
    print("Rephrased Question:", rephrased_question)
    
    code_prompt = build_code_prompt(rephrased_question)
    
    rephrase_results = []
    for s in range(NUM_SAMPLES_PER_REPHRASE):
        print(f"\n--- Sample {s+1} for Rephrase {r+1} ---")
        code = generate_code(code_prompt)
        print("Generated Code:\n", code, "\n")
        if code:
            recorded_outputs = run_generated_code(code, io_pair)
            rephrase_results.append({"code": code, "recorded_outputs": recorded_outputs})
        else:
            rephrase_results.append({"code": None, "recorded_outputs": []})
    final_results.append(rephrase_results)

# Save the results.
with open("final_results.json", "w") as f:
    json.dump(final_results, f, indent=4)

print("\nResults saved to final_results.json")


=== Testing on Original Question ===

Please generate a complete Python script that solves the following problem. Enclose your final code within ```python and ``` markers. Please ensure that your final code includes a main block to read input and print the result. Also, ensure that your entire output does not exceed 1024 tokens.

Problem Description:
You are given an array $a$ consisting of $n$ integers (it is guaranteed that $n$ is even, i.e. divisible by $2$). All $a_i$ does not exceed some integer $k$.

Your task is to replace the minimum number of elements (replacement is the following operation: choose some index $i$ from $1$ to $n$ and replace $a_i$ with some integer in range $[1; k]$) to satisfy the following conditions:  after all replacements, all $a_i$ are positive integers not greater than $k$;  for all $i$ from $1$ to $\frac{n}{2}$ the following equation is true: $a_i + a_{n - i + 1} = x$, where $x$ should be the same for all $\frac{n}{2}$ pairs of elements. 

You have to 

TypeError: _exit() missing required argument 'status' (pos 1)